我們終於要介紹三大神經網路的最後一個, 也就是 RNN。RNN 有不少的變型, 例如 LSTM 和 GRU 等等, 不過我們都通稱叫 RNN。RNN 是一種「有記憶」的神經網路, 非常適合時間序列啦, 或是不定長度的輸入資料。

我們來看看怎麼樣用 RNN 做電影評論的「情意分析」, 也就是知道一則評論究竟是「正評」還是「負評」。

## 09-01 初始準備

基本上和之前是一樣的, 我們就不再說明。

In [439]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [440]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping

## 09-02 讀入 IMDB 電影數據庫

今天我們要評入 IMDB 電影數據庫影評的部份。

In [441]:
from keras.datasets import imdb

In [442]:
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=10000)

要注意這裡我們限制只選「最常用」1 萬字, 也就是超過這範圍的就當不存在。這是文字分析常會做的事。

In [443]:
print('訓練總筆數：' , len(x_train))
print('測試總比數：',len(x_test))

訓練總筆數： 25000
測試總比數： 25000


### 輸入資料部份

我們來看一下輸入部份長什麼樣子?

In [444]:
# x_train[24999]

注意這其實是一個 list 而不是 array, 原因是每筆資料 (每段影評) 長度自然是不一樣的! 我們檢查一下前 10 筆的長度就可以知道。

In [445]:
len(x_train[24999])

153

In [446]:
len(x_train[9982])

156

In [447]:
len(x_train[9487])

104

最後要說明的是, 在每筆輸入資料的數字都代表英文的一個單字。編號方式是在我們資料庫裡所有文字的排序: 也就是出現頻率越高, 代表的數字就越小。

### 輸出資料部份

輸出方面應該很容易想像, 我們來看看前 10 筆。結果自然就是 0 (負評) 或 1 (正評)。

In [448]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [449]:
y_train[24999]

0

### 送入神經網路的輸入處理

雖然 RNN 是可以處理不同長度的輸入, 在寫程式時我們還是要

* 設輸入文字長度的上限
* 把每段文字都弄成一樣長, 太短的後面補上 0

In [450]:
from keras.preprocessing import sequence

In [451]:
x=300

In [452]:
x_train = sequence.pad_sequences(x_train , maxlen = x )
x_test = sequence.pad_sequences(x_test , maxlen= x)


In [453]:
x_train.shape

(25000, 300)

## 09-03 打造你的 RNN

這裡我們選用 LSTM, 基本上用哪種 RNN 寫法都是差不多的!

### 決定神經網路架構

* 先將 10000 維的文字壓到 N 維
* 然後用 K 個 LSTM 神經元做隱藏層
* 最後一個 output, 直接用 sigmoid 送出

### 建構我們的神經網路

文字我們用 1-hot 表示是很標準的方式, 不過要注意的是, 因為我們指定要 1 萬個字, 所以每個字是用 1 萬維的向量表示! 這一來很浪費記憶空間, 二來字和字間基本上是沒有關係的。我們可以用某種「合理」的方式, 把字壓到比較小的維度, 這些向量又代表某些意思 (比如說兩個字代表的向量角度小表相關程度大) 等等。

這聽來很複雜的事叫 "word embedding", 而事實上 Keras 會幫我們做。我們只需告訴 Keras 原來最大的數字是多少 (10000), 還有我們打算壓到幾維 (N)。

In [454]:
N = 128  # 文字壓縮到Ｎ維
K = 10  #ＬＳＴＭ有Ｎ個神經元


In [455]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,Dropout
from keras.layers import LSTM

In [456]:
model = Sequential()

In [457]:
model.add(Embedding(10000,N)) #輸入

In [458]:
#model.add(Dropout(0.2))

LSTM 層, 我們做 K 個 LSTM Cells。

In [459]:
model.add(LSTM(K))

In [460]:
#model.add(Dropout(0.2))

單純透過 sigmoid 輸出。

In [486]:

model.add(Dense(1,activation='sigmoid'))

### 組裝

這次我們用 binary_crossentropy 做我們的 loss function, 另外用一個很潮的 Adam 學習法。

In [480]:
model.compile(loss='binary_crossentropy',
             optimizer='Nadam',
             metrics=['acc'])

In [481]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, None, 128)         1280000   
_________________________________________________________________
lstm_17 (LSTM)               (None, 10)                5560      
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
_________________________________________________________________
dense_25 (Dense)             (None, 10)                20        
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 11        
_________________________________________________________________
dense_27 (Dense)             (None, 10)                20        
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 11        
Total para

In [482]:
## f神經元 LSTM = (K*7+K)*N

In [483]:
(4*(N+K)+4)*K

5560

## 09-04 訓練

我們用的 embedding 中, 會被 batch_size 影響輸入。輸入的 shape 會是

    (batch_size, 每筆上限)
    
也就是 (32,100) 輸出是 (32,100,128), 其中 128 是我們決定要壓成幾維的向量。

In [484]:
eralystop=EarlyStopping(monitor='val_acc', mode='auto',patience=0, verbose=2,restore_best_weights = True)

In [477]:
model.fit(x_train,y_train,
         batch_size=128,
         epochs=5)

Epoch 1/5
25000/25000 [==============================] - 108s 4ms/step - loss: 0.4549 - acc: 0.9131
Epoch 2/5
 6272/25000 [======>.......................] - ETA: 1:15 - loss: 0.2742 - acc: 0.9373

KeyboardInterrupt: 

In [485]:
model.fit(x_train,y_train,
         batch_size=128,
         epochs=5,
          validation_data=(x_test,y_test),
         callbacks=[eralystop])

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 100s 4ms/step - loss: 0.6949 - acc: 0.4986 - val_loss: 0.6932 - val_acc: 0.4999
Epoch 2/5
25000/25000 [==============================] - 102s 4ms/step - loss: 0.6933 - acc: 0.4951 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 3/5
25000/25000 [==============================] - 103s 4ms/step - loss: 0.6932 - acc: 0.4996 - val_loss: 0.6932 - val_acc: 0.5000
Restoring model weights from the end of the best epoch
Epoch 00003: early stopping


In [433]:
model.evaluate(x_test,y_test)

25000/25000 [==============================] - 29s 1ms/step


[0.39614982434272766, 0.86732]

## 09-05 檢視結果

### 分數

我們照例來看看測試資料的分數。

### 儲存結果

這裡有 8 成我們可以正確分辨, 看來還不差, 照例我們把結果存檔。